In [1]:
import datetime
import os
import praw
import queue
import requests
import threading

In [2]:
SUBREDDITS = ["earthporn", "spaceporn"]
MAX_IMG = 100
MAX_WORKERS=6
ACCEPTABLE_EXTENSIONS = ["jpg", "png"]
reddit = praw.Reddit("cs4243")

In [3]:
q = queue.Queue()
def get_image(url, filename) :
    req = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in req.iter_content(1024):
            if chunk:
                f.write(chunk)

def worker():
    while True:
        subr, id, url = q.get()
        get_image(url, f"./{subr.lower()}/{id}.jpg")
        q.task_done()

all_threads = [threading.Thread(target=worker) for _ in range(MAX_WORKERS)]
for t in all_threads:
    t.start()

In [4]:
now = datetime.datetime.now()
unixnow = int(datetime.datetime.timestamp(now))
datafile = open(f"crawl_{now.month:02}{now.day:02}.csv", 'w')
datafile.write("CS4243_crawler data\n")
datafile.write("subreddits\n" + '\n'.join(SUBREDDITS) + '\n\n')
for sr in SUBREDDITS:
    # os.makedirs(f"./{sr}_{now.month:02}{now.day:02}", exist_ok=True)
    os.makedirs(f"./{sr.lower()}", exist_ok=True)

sr_count = {}
for sr in SUBREDDITS:
    sr_count[sr.lower()] = 0
all_count = 0
all_limit = len(SUBREDDITS)*MAX_IMG
datafile.write("SUBREDDIT,ID,SCORE,URL,\n")
for submission in reddit.subreddit("+".join(SUBREDDITS)).new(limit=None):
    if (unixnow - submission.created_utc) > 604800:
        srname = submission.subreddit.display_name.lower()
        if sr_count[srname] >= MAX_IMG:
            continue
        if submission.url[-3:] not in ACCEPTABLE_EXTENSIONS:
            continue
        datafile.write(f"{srname},{submission.id},{submission.score},{submission.url}\n")
        q.put((srname, submission.id, submission.url))
        sr_count[srname] += 1
        all_count += 1

        if all_count % 10 == 0:
            datafile.flush()
            print(f"Sourcing {all_count}/{all_limit}")
            # break
        if all_count == all_limit:
            break
q.join()

Sourcing 10/200
Sourcing 20/200
Sourcing 30/200
Sourcing 40/200
Sourcing 50/200
Sourcing 60/200
Sourcing 70/200
Sourcing 80/200
Sourcing 90/200
Sourcing 100/200
Sourcing 110/200
Sourcing 120/200
Sourcing 130/200
Sourcing 140/200
Sourcing 150/200
Sourcing 160/200
Sourcing 170/200
Sourcing 180/200
Sourcing 190/200
Sourcing 200/200
